# Web Programming in Python

There are a variety of ways to utilize python to generate web content.  Here I will (briefly) go over two of the more popular ones, CGI and Flask.

## 1. Common Gateway Interface (CGI) 

The Common Gateway Interface (CGI) is a standard for external gateway programs to interface with information servers such as HTTP servers.  Essentially it indicates to the web server that it should not simply return the specified file, but instead treat it as an <i>executable</i> and return the output.  Note that CGI can handle many more languages than just python (perl, C, C++, bash, ...)

### Using python to generate HTML files

Imagine that you were so enamored with python that you wanted to use python scripts to generate <i>all</i> your HTML files for a given website.  Here's a trivial example:

In [ ]:
!cat cgi-bin/hello.cgi 

#### Exercise 1: Try to open hello.cgi in your web browser.  What happens?

To actually execute the file, you need a cgi-capable server running.  I've created an extremely simple one for you in this directory.

In [ ]:
!cat server.py

#### Exercise 2: Start the server and load the webpage (through the server)

From the command line, execute
<t>$ ./server.py</t>

Then open the following link in your web browser:
<t>http://localhost:8000/cgi-bin/hello.cgi</t>

Note that CGI files must be executable (though you can use other file extensions if desired).

In [ ]:
ls -lt cgi-bin

### Dynamic Content 

If this were all that was possible, it would not be very useful.  But the real power of python for CGI scripting lies in creating <i>dynamic</i> (as opposed to static) content.  We can nicely illustrate this with usage of the GET and POST Methods.  Here is a very simple example of how to parse and handle data from a form.

In [ ]:
!cat hello.html

In [ ]:
!cat cgi-bin/hello_get.cgi

To test this out, go to:
<t>http://localhost:8000/hello.html</t>

#### Exercise 3: Write a python CGI script to capture text from an HTML text form

To get you started, here's what the HTML page looks like.  You will need to write the textarea.cgi file.

In [ ]:
!cat text.html

<t>http://localhost:8000/text.html</t>

## 2. Flask 

Flask in a very popular "microframework" that allows you to generate web content in a more "pythonic" manner.  I will only have time to give a very brief introduction here.  To find out more detail, here is the <a href=http://flask.pocoo.org/>Flask website</a>.

### A (Brief) Aside on decorators

A <b>decorator</b> is a function that takes another function and extends the behavior of the latter function <i>without</i> explicitly modifying it.  This is easiest to see with an example.

In [ ]:
import time

def timing_function(some_function):

    """
    Outputs the time a function takes
    to execute.
    """

    def wrapper():
        t1 = time.time()
        some_function()
        t2 = time.time()
        return "Time it took to run the function: " + str((t2 - t1)) + "\n"
    return wrapper

In [ ]:
@timing_function
def my_function():
    num_list = []
    for num in (range(0, 10000)):
        num_list.append(num)
    print("\nSum of all the numbers: " + str((sum(num_list))))

In [ ]:
print(my_function())

Decorators are used extensively in Flask.

In [ ]:
ls

In [ ]:
cd flask

In [ ]:
!cat hello.py

This can be run directly from the command line:
<t> > setenv FLASK_APP hello.py # or for bash: FLASK_APP=hello.py
<t> > flask run

### minitwit: A Twitter clone (in 256 lines)

#### Step 0: Directory Structure

In [ ]:
ls

In [ ]:
cd minitwit

In [ ]:
ls

In [ ]:
cd minitwit

In [ ]:
ls

Pretty much all Flask sites are set up like this (as python <i>packages</i>).  Note that the <b>static</b> folder holds CSS / JavaScript templates.  Inside the <b>templates</b> folder, Flask will look for <a href=http://jinja.pocoo.org/docs/2.9/>Jinja2</a> templates.

In [ ]:
ls static

In [ ]:
ls templates

#### Step 1: Set up the database

Here we make use of the <i>sqlite3</i> package to handle the database.  <i>sqlite3</i> is small and powerful, but not portable.

In [ ]:
!cat schema.sql

#### Step 2: Application setup code

The first few code blocks in minitwit.py import the necessary libraries, configure and create the application.

In [ ]:
!sed -n '12,30p' minitwit.py

#### Step 3: Database connections

Creating and closing database connections all the time is very inefficient, so you will need to keep it around for longer. Because database connections encapsulate a transaction, you will need to make sure that only one request at a time uses the connection. An elegant way to do this is by utilizing the application context.

Flask provides two contexts: the application context and the request context. For the time being, all you have to know is that there are special variables that use these. For instance, the <i>request</i> variable is the request object associated with the current request, whereas <i>g</i> is a general purpose variable associated with the current application context.  For the time being, all you have to know is that you can store information safely on the <i>g</i> object.

So when do you put it on there? To do that you can make a helper function. The first time the function is called, it will create a database connection for the current context, and successive calls will return the already established connection:

In [ ]:
!sed -n '33,40p' minitwit.py

Now you know how to connect, but how can you properly disconnect? For that, Flask provides us with the <b>teardown_appcontext()</b> decorator. It’s executed every time the application context tears down.

In [ ]:
!sed -n '43,47p' minitwit.py

Functions marked with <b>teardown_appcontext()</b> are called every time the app context tears down. What does this mean? Essentially, the app context is created before the request comes in and is destroyed (torn down) whenever the request finishes. A teardown can happen because of two reasons: either everything went well (the error parameter will be None) or an exception happened, in which case the error is passed to the teardown function.

#### Step 4: Creating the Database

As outlined earlier, Minitwit is a database powered application, and more precisely, it is an application powered by a relational database system. Such systems need a schema that tells them how to store that information. Before starting the server for the first time, it’s important to create that schema.

Such a schema can be created by piping the schema.sql file into the sqlite3 command as follows:

$ sqlite3 /tmp/minitwit.db < schema.sql

The downside of this is that it requires the <i>sqlite3</i> command to be installed, which is not necessarily the case on every system. This also requires that you provide the path to the database, which can introduce errors. It’s a good idea to add a function that initializes the database for you, to the application.

To do this, you can create a function and hook it into a flask command that initializes the database. For now just take a look at the code segment below. A good place to add this function, and command, is just below the connect_db function in minitwit.py:

In [ ]:
!sed -n '50,62p' minitwit.py

The app.cli.command() decorator registers a new command with the flask script. When the command executes, Flask will automatically create an application context which is bound to the right application. Within the function, you can then access flask.g and other things as you might expect. When the script ends, the application context tears down and the database connection is released.

You will want to keep an actual function around that initializes the database, though, so that we can easily create databases in unit tests later on.

The open_resource() method of the application object is a convenient helper function that will open a resource that the application provides. This function opens a file from the resource location (the minitwit/minitwit folder) and allows you to read from it. It is used in this example to execute a script on the database connection.

The connection object provided by SQLite can give you a cursor object. On that cursor, there is a method to execute a complete script. Finally, you only have to commit the changes. SQLite3 and other transactional databases will not commit unless you explicitly tell it to.

Now, it is possible to create a database with the flask script:

$ flask initdb
Initialized the database.


#### Step 5: The View functions

Now that the database connections are working, you can start writing the view functions.  I will only show a few examples of these.

##### Timeline (default page)

In [ ]:
!sed -n '98,113p' minitwit.py

##### Login page

In [ ]:
!sed -n '193,211p' minitwit.py

##### Create a new message

In [ ]:
!sed -n '178,190p' minitwit.py

##### Follow another user

In [ ]:
!sed -n '146,159p' minitwit.py

#### Step 6: Create Templates

Now it is time to start working on the templates. As you may have noticed, if you make requests with the app running, you will get an exception that Flask cannot find the templates. The templates are using Jinja2 syntax and have autoescaping enabled by default. This means that unless you mark a value in the code with Markup or with the |safe filter in the template, Jinja2 will ensure that special characters such as < or > are escaped with their XML equivalents.

We are also using template inheritance which makes it possible to reuse the layout of the website in all pages.

It is generally advised to start with an overall template called "layout".

In [ ]:
!cat templates/layout.html

A simple extension of this is the register page:

In [ ]:
!cat templates/register.html

#### Step 7: Style Files

Finally, we can add CSS style files to control the overall look and feel of the site.

In [ ]:
!cat static/style.css